# Análisis Exploratorio de Datos sobre noticias de El Diario de Hoy

## Datos de referencia

En esta sección vamos a realizar un Análisis Exploratorio de Datos (EDA) a partir de la técnica de webscraping. En función de esto, se logró extraer una serie de variables para desarrollarlo. Entre las variables que pudimos obtener de la página de El Diario de Hoy fueron las siguientes:

*   Título
*   Resumen
*   Autor
*   Fecha
*   Noticia
*   Keywords
*   URL
*   Categoría










## Importación de librerias


In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.dates as mdates

In [2]:
#Leer el archivo CSV
df = pd.read_csv('DDH/combinado.csv')

## Preparar el DataFrame para el análisis

In [3]:
df.loc[df['Autor'] == 'PUBLICITY', 'Categoria'] = 'PUBLICITY'
df = df.drop_duplicates()
df = df[~df['Titulo'].str.contains('Foto EDH')]

## Análisis exploratorio


In [ ]:
#Muestra las primeras 5 filas
df.head(5)

In [ ]:
# Estadísticas descriptivas de columnas numéricas
print(df.describe())

In [ ]:
# Conteo de valores únicos en la columna 'Categoria'
print(df['Categoria'].value_counts()) 

In [ ]:
# Información general del DataFrame
print(df.info())

In [ ]:
# Forma de la Data
df.shape

In [ ]:
df.dtypes

## Gráficas

In [5]:
#Transformar la variable Fecha a un formato adecuado para el análisis e imprimir las primeras filas
df['Fecha'] = pd.to_datetime(df['Fecha'], format='%b %d, %Y- %H:%M', errors='coerce')
# print(df.head())

### Gráfico de lineas para la relación entre 'Fecha' y 'Noticia'

In [ ]:
# Crear un DataFrame agrupando por la fecha y contando el número de noticias por día
data_agrupada = df.groupby(df['Fecha'].dt.date).size()

# Crear y mostrar el gráfico de líneas
plt.figure(figsize=(8, 6))
plt.plot(data_agrupada.index, data_agrupada, marker='o', linestyle='-')

# Configuración de las etiquetas de las fechas
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.xticks(rotation=45, ha='right')  # Rotar las fechas para mejor visualización

# Configuración adicional del gráfico
plt.xlabel('Fecha')
plt.ylabel('Cantidad de Noticias')
plt.title('Distribución de Noticias a lo Largo del Tiempo')
plt.grid(True)

# Ajustar el layout y mostrar el gráfico
plt.tight_layout()  # Ajusta los parámetros del subplot para que encaje en el área del gráfico
plt.show()

### Gráfico de barras para la distribución de 'Dia_Semana'

In [ ]:
# Extraer el día de la semana (0 es lunes, 6 es domingo)
df['Dia_Semana'] = df['Fecha'].dt.dayofweek

# Crear una copia de los datos para evitar SettingWithCopyWarning
data_clean = df.dropna(subset=['Dia_Semana']).copy()

# Convertir 'Dia_Semana' a tipo entero
data_clean['Dia_Semana'] = data_clean['Dia_Semana'].astype(int)

# Agrupar por día de la semana y contar el número de noticias
conteo_dias_semana = data_clean.groupby('Dia_Semana').size()

# Crear un mapeo de números a nombres de días
dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
conteo_dias_semana.index = [dias[dia] for dia in conteo_dias_semana.index]

# Crear el gráfico de barras
sns.barplot(x=conteo_dias_semana.index, y=conteo_dias_semana.values)

# Etiquetas y título
plt.xlabel('Día de la Semana')
plt.ylabel('Cantidad de Noticias')
plt.title('Cantidad de Noticias por Día de la Semana')
plt.xticks(rotation=45)
plt.show()

### Gráfico Distribución por categorías

In [ ]:
# Drop rows with missing values in the 'Dia_Semana' and 'Categoria' columns
data_clean = df.dropna(subset=['Dia_Semana', 'Categoria']).copy()

# Convert 'Dia_Semana' to integer type
data_clean['Dia_Semana'] = data_clean['Dia_Semana'].astype(int)

# Group by both 'Dia_Semana' and 'Categoria' and count the occurrences
conteo_dias_categoria = data_clean.groupby(['Dia_Semana', 'Categoria']).size().unstack()

# Map the numerical day of the week to the corresponding day name
dias = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
conteo_dias_categoria.index = [dias[dia] for dia in conteo_dias_categoria.index]

# Plot the data using a heatmap
sns.heatmap(conteo_dias_categoria, cmap='viridis', annot=True, fmt='g', cbar_kws={'label': 'Cantidad de Noticias'})
plt.xlabel('Categoría')
plt.ylabel('Día de la Semana')
plt.title('Cantidad de Noticias por Día de la Semana y Categoría')
plt.show()